In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import plotly.express as px

from sklearn.feature_extraction import DictVectorizer

#### Q1. Downloading the data

Read the data for January. How many records are there?

In [2]:
df = pd.read_parquet("./data/fhv_tripdata_2021-01.parquet")

In [3]:
df.shape

(1154112, 7)

#### Q2. Computing duration

Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the average trip duration in January?

In [4]:
df['duration'] = df.dropOff_datetime - df.pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [5]:
df.duration.mean()

19.167224093791006

In [6]:
df.groupby(by=[pd.DatetimeIndex(df['dropOff_datetime']).month]).duration.mean()

dropOff_datetime
1         18.755341
2        105.907463
3      40347.600000
11    423371.050000
Name: duration, dtype: float64

Answer is `19.16`, but real answer is `18.755341`

#### Data preparation

Check the distribution of the duration variable. There are some outliers.

Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

How many records did you drop?

In [7]:
df  = df[['PUlocationID', 'DOlocationID', 'duration']]

In [8]:
df = df[(df["duration"] >= 1) & (df["duration"] <= 60)]

In [9]:
df.describe()[['duration']]

,duration
count,1.109826e+06
mean,1.624725e+01
std,1.155150e+01
min,1.000000e+00
25%,7.850000e+00
50%,1.323333e+01
75%,2.146667e+01
max,6.000000e+01


In [10]:
df.shape

(1109826, 3)

Answer is `1154112 - 1109826 = 44286`

#### Q3. Missing values

The features we'll use for our model are the pickup and dropoff location IDs.

But they have a lot of missing values there. Let's replace them with "-1".

What's the fractions of missing values for the pickup location ID? I.e. fraction of "-1"s after you filled the NAs.

In [11]:
df = df.fillna(-1)

In [12]:
df[df['PUlocationID'] == -1].shape[0] / df.shape[0]

0.8352732770722617

Answer is `83%`

#### Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

- Turn the dataframe into a list of dictionaries
- Fit a dictionary vectorizer
- Get a feature matrix from it

What's the dimensionality of this matrix? (The number of columns).

In [13]:
categorical = ['PUlocationID', 'DOlocationID']
df_dicts = df[categorical].to_dict(orient='records')
df_dicts

[{'PUlocationID': -1.0, 'DOlocationID': -1.0},
 {'PUlocationID': -1.0, 'DOlocationID': -1.0},
 {'PUlocationID': -1.0, 'DOlocationID': 72.0},
 {'PUlocationID': -1.0, 'DOlocationID': 61.0},
 {'PUlocationID': -1.0, 'DOlocationID': 71.0},
 {'PUlocationID': -1.0, 'DOlocationID': 91.0},
 {'PUlocationID': -1.0, 'DOlocationID': 39.0},
 {'PUlocationID': -1.0, 'DOlocationID': 37.0},
 {'PUlocationID': -1.0, 'DOlocationID': 39.0},
 {'PUlocationID': -1.0, 'DOlocationID': 72.0},
 {'PUlocationID': -1.0, 'DOlocationID': 72.0},
 {'PUlocationID': -1.0, 'DOlocationID': 89.0},
 {'PUlocationID': -1.0, 'DOlocationID': 177.0},
 {'PUlocationID': -1.0, 'DOlocationID': 225.0},
 {'PUlocationID': -1.0, 'DOlocationID': 63.0},
 {'PUlocationID': -1.0, 'DOlocationID': 67.0},
 {'PUlocationID': -1.0, 'DOlocationID': 22.0},
 {'PUlocationID': -1.0, 'DOlocationID': 61.0},
 {'PUlocationID': -1.0, 'DOlocationID': 14.0},
 {'PUlocationID': -1.0, 'DOlocationID': 14.0},
 {'PUlocationID': -1.0, 'DOlocationID': 188.0},
 {'PUlocat

In [14]:
dv = DictVectorizer()

X_train = dv.fit_transform(df_dicts)
X_train

<1109826x2 sparse matrix of type '<class 'numpy.float64'>'
	with 2219652 stored elements in Compressed Sparse Row format>

In [15]:
X_train.shape

(1109826, 2)

In [16]:
pd.DataFrame(X_train.toarray())

,0,1
0,-1.0,-1.0
1,-1.0,-1.0
2,72.0,-1.0
3,61.0,-1.0
4,71.0,-1.0
...,...,...
1109821,7.0,7.0
1109822,91.0,44.0
1109823,171.0,171.0
1109824,15.0,15.0


In [17]:
X_train.toarray()

array([[ -1.,  -1.],
       [ -1.,  -1.],
       [ 72.,  -1.],
       ...,
       [171., 171.],
       [ 15.,  15.],
       [ -1.,  -1.]])